Langchain 을 활용한 모델 사용,비용 모니터링 및 캐싱 전략
    GPT-4o-mini  GPT-3.5-Turbo 비용이 60% 저렴
    LangChain V0.3x 부터 openAi가 별도 패키지로 분리 필요 패키지를 설치 langchain-openai 필요 
    토큰사용량 추적, 캐싱을 위한 langchain-community도 별도설치
    환경변수 변수 관리 패키지 python-dotevn

In [1]:
%pip install langchain-openai langchain-community python-dotenv openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0.7)  # dotenv 를 하면 자동으로 OPENAI_API_KEY 참조
prompt = 'LangChain에 대해 한 문장으로 설명해줘'
result = llm.invoke(prompt)
result.content

'LangChain은 다양한 언어 모델을 연결하고 구성하여 복잡한 자연어 처리 응용 프로그램을 구축할 수 있도록 돕는 프레임워크입니다.'

In [4]:
#사용량
result.usage_metadata

{'input_tokens': 18,
 'output_tokens': 35,
 'total_tokens': 53,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [5]:
# 콜벡함수를 통한 누적 토큰 추적(get_openai_callback)
from langchain_community.callbacks import get_openai_callback
with get_openai_callback() as cb:
    # 첫번째 호출
    res1 = llm.invoke('서울의 오늘 날씨는 어떤지 알려줘?')
    print('응답1', res1.content[:10],'...')
    # 두번째 호출
    res2 = llm.invoke('파이썬으로 랭체인 사용법을 알려줘')
    print('응답2', res2.content[:10],'...')

# 누적 토큰 사용량 출력  콜백 cb에는 블록 내 전체 토큰 사용량이 누적
# 총 토큰수
print('총 토큰수:', cb.total_tokens)
# 프롬프트 토큰수
print('프롬프트 토큰수:', cb.prompt_tokens)
# 응답 토큰수
print('응답 토큰수:', cb.completion_tokens)
# 비용 계산
print('비용(USD):', cb.total_cost)

응답1 죄송하지만, 실시간 ...
응답2 LangChain은 ...
총 토큰수: 676
프롬프트 토큰수: 39
응답 토큰수: 637
비용(USD): 0.0003880499999999999


In [6]:
# LangChain의 LLM 응답캐싱 (InMemory Cache, SQLiteCache)
# 동일한 질문은 저장해 뒀다가 응답에 사용
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# InMemoryCash 설정
set_llm_cache(InMemoryCache())

In [7]:
# 캐시 사용 전후를 비교, 같은 질문을 두번 호출
query = "재미있는 유머 하나 알려줘"
# 첫 번째 호출(캐시에 없으면 api 호출 발생)
result1 = llm.invoke(query)
print(f'응답1 : {result1.content}')
print('*'*100)
# 두 번째 호출(동일한 query, 캐시를 확인하고 동일 질문이면 api 미호출)
query = "재미있는 유머 하나 알려줘"
result2 = llm.invoke(query)
print(f'응답2 : {result2.content}')

응답1 : 물론이죠! 다음은 간단한 유머입니다:

왜 컴퓨터는 바다를 싫어할까요?

바다에 바이러스가 많아서요! 😄

재미있으셨나요? 더 듣고 싶으시면 언제든지 말씀해 주세요!
****************************************************************************************************
응답2 : 물론이죠! 다음은 간단한 유머입니다:

왜 컴퓨터는 바다를 싫어할까요?

바다에 바이러스가 많아서요! 😄

재미있으셨나요? 더 듣고 싶으시면 언제든지 말씀해 주세요!


In [8]:
# 실행시간 측정
import time
# 첫 번째 호출 시간
query = '점심메뉴 추천해줘'
start = time.time(); llm.invoke(query); end = time.time()
print(f'첫번째 호출시간 : {end-start}')

start = time.time(); llm.invoke(query); end = time.time()
print(f'두번째 호출시간 : {end-start}')

첫번째 호출시간 : 5.322176694869995
두번째 호출시간 : 0.0007736682891845703


In [9]:
# SQLite 캐시(디스크기반 캐시)
import os, time
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini',temperature=0.7) 

# SQLite 캐시 설정(지정한 경로의 DB 파일을 생성 / 사용)    
set_llm_cache(SQLiteCache(database_path='.langchain2.db'))  # langchain.db있으면 사용 없으면 생성

# 동일한 query를 두번 호출해서 결과와 시간을 비교
query = '썰렁한 유머 하나 알려줘'
# 첫 번째 호출(캐시에 없으면 api 호출 발생)
start = time.time(); result1 = llm.invoke(query); end = time.time()
print(f'첫번째 호출시간 : {end-start}')
print(f'응답1 : {result1.content}')

# 두 번째 호출(동일한 query, 캐시를 확인하고 동일 질문이면 api 미호출)
start = time.time(); result2 = llm.invoke(query); end = time.time()
print(f'두번째 호출시간 : {end-start}')
print(f'응답2 : {result2.content}')

첫번째 호출시간 : 1.8946354389190674
응답1 : 왜 바나나는 항상 혼자일까요?

왜냐하면, 껍질이 벗겨지면 슬퍼지니까요! 🍌😄
두번째 호출시간 : 0.002530336380004883
응답2 : 왜 바나나는 항상 혼자일까요?

왜냐하면, 껍질이 벗겨지면 슬퍼지니까요! 🍌😄
